In [1]:
#@title INSTALLATIONS
%%capture
!pip install -q -U transformers==4.37.2
!pip install sacremoses
!pip install gradio

In [23]:
!pip install rouge

In [2]:
from pycocotools.coco import COCO
from nltk.translate.bleu_score import corpus_bleu


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

import os
import json
import torch
import random
#from lavis.models import load_model_and_preprocess
from PIL import Image

import warnings
warnings.filterwarnings("ignore")

In [5]:
#@title BLIP IMAGE CAPTIONING MODEL
from transformers import AutoProcessor, BlipForConditionalGeneration

blip_processor = AutoProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [6]:
def blip_inference(visual):
  inputs = blip_processor(images=visual, text="an image of ",return_tensors="pt")
  outputs = blip_model.generate(**inputs)
  visual_prompt = blip_processor.decode(outputs[0], skip_special_tokens=True)
  return visual_prompt

In [7]:
#@title GET THE VIT-GPT2 IMAGE CAPTIONING
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
vit_model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
vit_feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
vit_tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [8]:
def vit_inference(visual):
  pixel_values = vit_feature_extractor(visual, return_tensors="pt").pixel_values
  generated_ids = vit_model.generate(pixel_values)
  return vit_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [9]:
#@title GIT MODEL
from transformers import AutoProcessor, AutoModelForCausalLM
git_processor = AutoProcessor.from_pretrained("microsoft/git-base")
git_model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")

preprocessor_config.json:   0%|          | 0.00/503 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/707M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [10]:
def git_inference(visual):
  pixel_values = git_processor(images=visual, return_tensors="pt").pixel_values
  generated_ids = git_model.generate(pixel_values=pixel_values, max_length=50)
  generated_caption = git_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
  return generated_caption

In [ ]:
#@title LLAVA Model
from transformers import pipeline

model_id = "llava-hf/llava-1.5-7b-hf"

pipe = pipeline("image-to-text", model=model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def llava_inference(visual,prompt="USER: <image>\nWhat are these?\nASSISTANT:"):

  outputs = pipe(visual, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
  caption = outputs[0]["generated_text"].split('ASSISTANT:')[1].strip().replace('\n', '')
  return caption

In [ ]:
#@title Image
from PIL import Image
raw_image = Image.open("burger.jpg").convert("RGB")

In [ ]:
#@title Dataset Creation
!unzip ubereat_2023.zip

In [12]:
import os

def list_files(folder_path):
    files = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            files.append(file_name)
    return files

#Image paths
Image_file_paths = []
main_titles = ['californiaroll','frenchfries','garlicnaan','misosoup','padthai']
folder_path = 'ubereat_2023/'
for title in main_titles:
  full_folder_path = folder_path + title
  files = list_files(full_folder_path)
  for file in files:
    Image_file_paths.append(os.path.join(full_folder_path, file))

In [ ]:
eval_df = pd.DataFrame()
eval_df['image_path'] = Image_file_paths
eval_df

In [15]:
eval_df.to_csv('ubereat_2023_eval.csv', index=False)

In [16]:
eval_df = pd.read_csv('ubereat_2023_eval_captions.csv')
eval_df.head()

,image_path,captions
0,ubereat_2023/californiaroll/californiaroll_2.jpg,a plate of california sushi rolls
1,ubereat_2023/californiaroll/californiaroll_4.jpg,a plate of california sushi rolls
2,ubereat_2023/californiaroll/californiaroll_6.jpg,a plate of california sushi rolls
3,ubereat_2023/californiaroll/californiaroll_1.jpg,a single california sushi roll with avocado an...
4,ubereat_2023/californiaroll/californiaroll_8.jpg,two california sushi rolls


In [17]:
#@title EVALUATION
from tqdm import tqdm
for index, row in tqdm(eval_df.iterrows()):
  image = Image.open(row['image_path']).convert("RGB")
  blip_caption = blip_inference(image)
  vit_caption = vit_inference(image)
  git_caption = git_inference(image)
  eval_df.loc[index, 'blip_caption'] = blip_caption
  eval_df.loc[index, 'vit_caption'] = vit_caption
  eval_df.loc[index, 'git_caption'] = git_caption


0it [00:00, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.
53it [04:40,  5.29s/it]


In [18]:
eval_df.to_csv('ubereat_2023_eval.csv', index=False)

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu

# Function to calculate BLEU score
def calculate_bleu_score(ai_caption, human_caption):
    ai_caption_tokenized = ai_caption.split()
    human_caption_tokenized = human_caption.split()
    return corpus_bleu([[human_caption_tokenized]], [ai_caption_tokenized])

# Apply the function to calculate BLEU scores for each row
eval_df['bleu_score_BLIP'] = eval_df.apply(lambda row: calculate_bleu_score(row['blip_caption'], row['captions']), axis=1)
eval_df['bleu_score_VIT'] = eval_df.apply(lambda row: calculate_bleu_score(row['vit_caption'], row['captions']), axis=1)
eval_df['bleu_score_GIT'] = eval_df.apply(lambda row: calculate_bleu_score(row['git_caption'], row['captions']), axis=1)
eval_df.head(20)


In [24]:
eval_df['bleu_score_BLIP'].mean()

0.016235323663425595

In [25]:
eval_df['bleu_score_VIT'].mean()

2.174183688284134e-78

In [26]:
eval_df['bleu_score_GIT'].mean()

0.008695540371537011

## INFERENCE FOR THE THREE MODELS

In [ ]:
order_food_prompt = """You are an AI that can understand images and make deep analysis base on
    the visual description of current scenario.
    You can place orders online.
  Example:
  Human: Order this food from doordash
  Visual:image of kung pao chicken
  AI: Sure, I can do it for you. [order kung pao chicken from doordash]

  Order food can be done with something like [order <item> from Amazon], one action square bracket.



  Based on the image uploaded and the example above answer the following:

  Human:Order this food from doordash
  Visual: ...
  AI: ...

  """

In [ ]:
import gradio as gr

## generate the captions from the 3 models
def generate_captions(image):
    blip_caption = blip_inference(image)
    vit_caption = vit_inference(image)
    git_caption = git_inference(image)
    llava_caption = llava_inference(image)
    return blip_caption, vit_caption, git_caption, llava_caption

# Create Gradio interface
inputs =  gr.Image(type='pil',label="Input Image")
outputs = [gr.Textbox(label="Caption from BLIP-BASE"),
           gr.Textbox(label="Caption from VIT-GPT2-BASE"),
           gr.Textbox(label="Caption from GIT-BASE"),
           gr.Textbox(label="Caption from LLAVA-1.5")]

gr.Interface(fn=generate_captions, inputs=inputs, outputs=outputs, title="Image Captioning").launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e1818989613a220280.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e1818989613a220280.gradio.live
